In [1]:
from pydantic import BaseModel
from typing import List, Dict, Literal
import httpx


class QueryVectorDatabase(BaseModel):
    collection_name: Literal[
    "thong_tin_truong_dai_hoc",   # thông tin về trường đại học
    "thong_tin_khoa_cong_nghe_thong_tin",  # thông tin về khoa công nghệ thông tin
    "thong_tin_khoa_ngon_ngu",  # thông tin về khoa ngôn ngữ
    "thong_tin_khoa_kinh_te",  # thông tin về khoa kinh tế
    "thong_tin_khoa_y",  # thông tin về khoa y
    "thong_tin_khoa_cong_nghe_sinh_hoc",  # thông tin về khoa công nghệ sinh học
    "thong_tin_khoa_dieu_duong",  # thông tin về khoa điều dưỡng
    "thong_tin_khoa_khai_phong",  # thông tin về khoa khai phóng
    "thong_tin_giang_vien",    # thông tin về giảng viên
    "thong_tin_nghien_cuu",     # thông tin về nghiên cứu
    "thong_tin_chi_phi",     # thông tin về tất cả chi phí (học phí, nợ môn, thi lại, v.v.)
    ]
    content: str 
    vector: List[float]
    limit: int

vectordb_api_host = '0.0.0.0'
vectordb_api_port = '8000'
async def query_vectordb(request: QueryVectorDatabase):
    url = f"http://{vectordb_api_host}:{vectordb_api_port}/retriever/query"
    
    payload = request.model_dump()
    
    headers = {
        'Content-Type': 'application/json'
    }
    
    async with httpx.AsyncClient() as client:
        timeout = httpx.Timeout(connect=5.0, read=60.0, write=10.0, pool=10.0)
        response = await client.post(url, headers=headers, json=payload, timeout=timeout)
        response.raise_for_status()
    
    return response.json()

In [2]:
from langchain_core.tools import tool


@tool
async def search_at_thong_tin_truong_dai_hoc(query: str, vector: List[float]) -> Dict:
    """
    Search for relevant documents in the "thong_tin_truong_dai_hoc" collection using a query string and a vector.

    Args:
        query (str): The textual query to search for within the collection.
        vector (List[float]): The vector representation of the query for vector-based search.

    Returns:
        Dict: The search results retrieved from the vector database, containing relevant documents.
    """
    return await query_vectordb(QueryVectorDatabase(collection_name="thong_tin_truong_dai_hoc", content=query, vector=vector, limit=7))
